In [15]:
#limpa df, comeca etc

import pandas as pd
import numpy as np
import re

df = pd.read_csv('data/IMDB.csv' , encoding='utf-8')

def clean_t(t):
    t_clean = re.sub(r'<.*?>', '', t)
    t_clean = re.sub(r'[^\w\s]', '', t_clean)
    return t_clean

df['review'] = df['review'].apply(clean_t)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [64]:
#fazendo o treino
df_treino = df.iloc[:25]
df_treino.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [66]:
#fazendo a dict de palavras nao repetidas


# Inicializa os dicionários de contagem para positivos e negativos
contagem_palavras = {}

# Itera sobre os textos e sentimentos
for texto, sentimento in zip(df_treino['review'], df_treino['sentiment']):
    palavras = texto.lower().split()  # Divide o texto em palavras
    for palavra in palavras:
        if palavra not in contagem_palavras:
            contagem_palavras[palavra] = {'positive': 0, 'negative': 0}
        contagem_palavras[palavra][sentimento] += 1

# Inicializa os dicionários de probabilidade para positivos e negativos
probabilidades_palavras = {}

# Calcula as probabilidades
for palavra, contagem in contagem_palavras.items():
    total_positivo = contagem['positive']
    total_negativo = contagem['negative']
    
    # Calcula as probabilidades
    prob_positivo = total_positivo / (total_positivo + total_negativo)
    prob_negativo = total_negativo / (total_positivo + total_negativo)
    
    # Adiciona as probabilidades ao dicionário
    probabilidades_palavras[palavra] = {'positive': prob_positivo, 'negative': prob_negativo}

print(probabilidades_palavras)

{'one': {'positive': 0.5161290322580645, 'negative': 0.4838709677419355}, 'of': {'positive': 0.48672566371681414, 'negative': 0.5132743362831859}, 'the': {'positive': 0.4166666666666667, 'negative': 0.5833333333333334}, 'other': {'positive': 0.5714285714285714, 'negative': 0.42857142857142855}, 'reviewers': {'positive': 1.0, 'negative': 0.0}, 'has': {'positive': 0.5555555555555556, 'negative': 0.4444444444444444}, 'mentioned': {'positive': 1.0, 'negative': 0.0}, 'that': {'positive': 0.32608695652173914, 'negative': 0.6739130434782609}, 'after': {'positive': 0.6, 'negative': 0.4}, 'watching': {'positive': 0.8, 'negative': 0.2}, 'just': {'positive': 0.35294117647058826, 'negative': 0.6470588235294118}, '1': {'positive': 1.0, 'negative': 0.0}, 'oz': {'positive': 1.0, 'negative': 0.0}, 'episode': {'positive': 1.0, 'negative': 0.0}, 'youll': {'positive': 1.0, 'negative': 0.0}, 'be': {'positive': 0.5, 'negative': 0.5}, 'hooked': {'positive': 1.0, 'negative': 0.0}, 'they': {'positive': 0.5, '

In [14]:
df_teste = df.iloc[25000:]
df_teste.head()

,review,sentiment
25000,This movie was bad from the start The only pur...,negative
25001,God I never felt so insulted in my whole life ...,negative
25002,Not being a fan of the Coen Brothers or George...,positive
25003,The movie Andaz Apna Apna in my books is the t...,positive
25004,I have to say I was really looking forward on ...,negative
